# RBF NN from scratch

- https://towardsdatascience.com/most-effective-way-to-implement-radial-basis-function-neural-network-for-classification-problem-33c467803319 

In [147]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/home/sebacastillo/neuralnets/')
from src.utils import get_project_root
root = get_project_root()
from scipy.spatial import distance
from sklearn.model_selection import train_test_split

In [148]:
# Function to compute Euclidean distance
def get_distance(c, x):
    return distance.euclidean(c, x)

def kmeans(X, k, max_iters):
    """
    Performs k-means clustering.

    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        The input samples.
    k : int
        The number of clusters.
    max_iters : int
        Maximum number of iterations.

    Returns
    -------
    centroids : array, shape (k, n_features)
        The final centroids.
    std_devs : list
        The standard deviation of each cluster.
    """

    # Step 1: Initialize the centroids randomly from the data points
    centroids = X[np.random.choice(range(len(X)), k, replace=False)]
    
    converged = False
    current_iter = 0

    # Step 2: Run the main k-means algorithm
    while (not converged) and (current_iter < max_iters):

        # Create empty list for each cluster
        cluster_list = [[] for i in range(len(centroids))]

        # Go through each data point
        for x in X:  
            distances_list = []

            # Compute distance of 'x' from each centroid
            for c in centroids:
                distances_list.append(get_distance(c, x))

            # Assign 'x' to closest centroid
            cluster_list[int(np.argmin(distances_list))].append(x)

        # Remove empty clusters
        cluster_list = list((filter(None, cluster_list)))

        # Deep copy of the current centroids for convergence check later
        prev_centroids = centroids.copy()

        centroids = []

        # Step 3: Update each centroid to the mean of points in its cluster
        for j in range(len(cluster_list)):
            centroids.append(np.mean(cluster_list[j], axis=0))

        # Step 4: Check for convergence
        # (if the centroids haven't moved, we're done)
        pattern = np.abs(np.sum(prev_centroids) - np.sum(centroids))

        print('K-MEANS: ', pattern)

        converged = (pattern == 0)

        current_iter += 1

    # Return final centroids, and standard deviation of each cluster
    return np.array(centroids), [np.std(x) for x in cluster_list]


In [149]:
#data = np.load(str(root) + '/data/mnist_data.npy').astype(float)
data = pd.read_csv(str(root) + '/data/concentlite.csv', header=0).to_numpy()
data.shape

(832, 3)

In [150]:
X = data[:, :-1]
k = 2

In [151]:
kmeans(X, k, 100)

K-MEANS:  0.35598725488826766
K-MEANS:  0.21401944134925532
K-MEANS:  0.10265381904309345
K-MEANS:  0.027114187999586425
K-MEANS:  0.029365872969709272
K-MEANS:  0.007376131803113806
K-MEANS:  0.0008021652716827532
K-MEANS:  0.006178629518859502
K-MEANS:  0.009266024477504686
K-MEANS:  0.010475872233983718
K-MEANS:  0.008155361307220055
K-MEANS:  0.011682589773487795
K-MEANS:  0.0026273189709566935
K-MEANS:  0.004813386230330785
K-MEANS:  4.884961888995676e-05
K-MEANS:  0.0007630696835105866
K-MEANS:  0.0


(array([[0.55961816, 0.30189673],
        [0.45188827, 0.72886926]]),
 [0.23107856338425442, 0.23654167408275018])

In [152]:
class RBF:

    def __init__(self, X, y, tX, ty, num_of_classes,
                 k, std_from_clusters=True):
        self.X = X
        self.y = y

        self.tX = tX
        self.ty = ty

        self.number_of_classes = num_of_classes
        self.k = k
        self.std_from_clusters = std_from_clusters

    def convert_to_one_hot(self, x, num_of_classes):
        arr = np.zeros((len(x), num_of_classes))
        for i in range(len(x)):
            c = int(x[i])
            arr[i][c] = 1
        return arr

    def rbf(self, x, c, s):
        distance = get_distance(x, c)
        return 1 / np.exp(-distance / s ** 2)

    def rbf_list(self, X, centroids, std_list):
        RBF_list = []
        for x in X:
            RBF_list.append([self.rbf(x, c, s) for (c, s) in zip(centroids, std_list)])
        return np.array(RBF_list)
    
    def fit(self):

        self.centroids, self.std_list = kmeans(self.X, self.k, max_iters=1000)

        if not self.std_from_clusters:
            dMax = np.max([get_distance(c1, c2) for c1 in self.centroids for c2 in self.centroids])
            self.std_list = np.repeat(dMax / np.sqrt(2 * self.k), self.k)

        RBF_X = self.rbf_list(self.X, self.centroids, self.std_list)

        self.w = np.linalg.pinv(RBF_X.T @ RBF_X) @ RBF_X.T @ self.convert_to_one_hot(self.y, self.number_of_classes)

        RBF_list_tst = self.rbf_list(self.tX, self.centroids, self.std_list)

        self.pred_ty = RBF_list_tst @ self.w

        self.pred_ty = np.array([np.argmax(x) for x in self.pred_ty])

        diff = self.pred_ty - self.ty

        print('Accuracy: ', len(np.where(diff == 0)[0]) / len(diff))

In [153]:
X_train, X_test = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)  # 80% train - 20% test

train_x = X_train[:,:-1]
train_y = X_train[:, -1]
test_x = X_test[:,:-1]
test_y = X_test[:, -1]

In [154]:
RBF_CLASSIFIER = RBF(train_x, train_y, test_x, test_y, num_of_classes=2,
                    k=2, std_from_clusters=False)

In [158]:
RBF_CLASSIFIER.fit()

K-MEANS:  0.2526280194075077
K-MEANS:  0.0404855762246763
K-MEANS:  0.026730718551489563
K-MEANS:  0.015891313547805375
K-MEANS:  0.04481242024612442
K-MEANS:  0.02552519375879081
K-MEANS:  0.014933725507408457
K-MEANS:  0.022646418708979876
K-MEANS:  0.02473271737168581
K-MEANS:  0.010960796295592434
K-MEANS:  0.007548659348658671
K-MEANS:  0.0012201575431487477
K-MEANS:  0.01831271261580114
K-MEANS:  0.005620857495008513
K-MEANS:  0.0010314778559585847
K-MEANS:  0.012480683415249327
K-MEANS:  0.016035326797346983
K-MEANS:  0.011489214255194113
K-MEANS:  0.00137615606676178
K-MEANS:  0.004493180173426037
K-MEANS:  0.0019728484172785166
K-MEANS:  0.0037705751312135938
K-MEANS:  0.004466679517950212
K-MEANS:  0.0
Accuracy:  0.6167664670658682
